# Recommendation System Datasets

 - MovieLens 10M data set
 - MovieLens 22M data set
 - Million song data set

# Split dataset into 60-20-20 train-validate-test partitions

In [25]:
import os

def exists(filepath):
    return os.path.exists(filepath)

In [43]:
# show current files
!ls -l ml-10M100K/

total 1043656
-rw-r--r--@ 1 fnokeke  staff      11563 Jan 29 10:38 README.html
-rwxr-x---@ 1 fnokeke  staff        753 Jan  5  2009 allbut.pl
-rw-r--r--@ 1 fnokeke  staff     522197 Jan  5  2009 movies.dat
-rw-r--r--@ 1 fnokeke  staff  265105635 Jan  5  2009 ratings.dat
-rwxr-x---@ 1 fnokeke  staff       1304 Feb 16 10:06 split_ratings.sh
-rw-r--r--@ 1 fnokeke  staff    3584119 Jan  5  2009 tags.dat
-rw-r--r--  1 fnokeke  staff   51584300 Apr  6 11:43 test20.dat
-rw-r--r--  1 fnokeke  staff  161529860 Apr  6 11:43 train60.dat
-rw-r--r--  1 fnokeke  staff   51990078 Apr  6 11:43 validation20.dat


In [44]:
if (exists('ml-10M100K/train60.dat') and exists('ml-10M100K/validation20.dat') and exists('ml-10M100K/test20.dat')):
    print "Already created files: train60.dat, validation20.dat, test20.dat"    

else:
    # sort by timestamp (4th column)
    print 'sorting file...'
    !sort -t ':' -k4 ml-10M100K/ratings.dat > ml-10M100K/new_ratings.dat 
    print "sorting complete."
    
    # split into 5 parts of 2 million each: train(3 parts), validation (1 part), test (1 part)
    print "splitting file..."
    !split -l 2000000 ml-10M100K/new_ratings.dat ff
    !cat ffaa ffab ffac > ml-10M100K/train60.dat
    !mv ffad ml-10M100K/validation20.dat
    !mv ffae ml-10M100K/test20.dat
    
    # remove tmp files used to create partitions
    !rm new_ratings.dat ff*
    print "splitting complete."    
    print "Newly created files: train60.dat, validation20.dat, test20.dat"

Already created files: train60.dat, validation20.dat, test20.dat


# Using train data, learn ALS model

# Using validation data, choose different regularization parameters with different latent factors

# Using test data, test chosen model and report metric error